In [3]:
import numpy as np

In [4]:
import csv
file_path = r"A:\Projects\Sanskrit Text Conversion\Sanskrit-Text-Conversion\Gita-data.csv"
with open(file_path, "r") as f:
    csvFile = csv.DictReader(f)
    for lines in csvFile:
        print(lines)

{'id': 'c:1v1', 'input': 'धृतराष्ट्र उवाच |धर्मक्षेत्रे कुरुक्षेत्रे समवेता युयुत्सवः |मामकाः पाण्डवाश्चैव किमकुर्वत सञ्जय ||', 'output': 'Dhritarashtra said: O Sanjay, after gathering on the holy field of Kurukshetra, and desiring to fight, what did my sons and the sons of Pandu do?'}
{'id': 'c:1v2', 'input': 'सञ्जय उवाच ।दृष्ट्वा तु पाण्डवानीकं व्यूढं दुर्योधनस्तदा ।आचार्यमुपसङ्गम्य राजा वचनमब्रवीत् ।। ।।', 'output': 'Sanjay said: On observing the Pandava army standing in military formation, King Duryodhan approached his teacher Dronacharya, and said the following words.'}
{'id': 'c:1v3', 'input': 'पश्यैतां पाण्डुपुत्राणामाचार्य महतीं चमूम् ।व्यूढां द्रुपदपुत्रेण तव शिष्येण धीमता ।। ।।', 'output': 'Duryodhan said: Respected teacher! Behold the mighty army of the sons of Pandu, so expertly arrayed for battle by your own gifted disciple, the son of Drupad.'}
{'id': 'c:1v4', 'input': 'अत्र शूरा महेष्वासा भीमार्जुनसमा युधि ', 'output': 'Behold in their ranks are many powerful warriors, l

In [5]:
import pandas as pd
csvFile = pd.read_csv(file_path)
csvFile = csvFile.drop("id", axis="columns")
print(csvFile.head(5))

                                               input  \
0  धृतराष्ट्र उवाच |धर्मक्षेत्रे कुरुक्षेत्रे समव...   
1  सञ्जय उवाच ।दृष्ट्वा तु पाण्डवानीकं व्यूढं दुर...   
2  पश्यैतां पाण्डुपुत्राणामाचार्य महतीं चमूम् ।व्...   
3             अत्र शूरा महेष्वासा भीमार्जुनसमा युधि    
4             अत्र शूरा महेष्वासा भीमार्जुनसमा युधि    

                                              output  
0  Dhritarashtra said: O Sanjay, after gathering ...  
1  Sanjay said: On observing the Pandava army sta...  
2  Duryodhan said: Respected teacher! Behold the ...  
3  Behold in their ranks are many powerful warrio...  
4  Behold in their ranks are many powerful warrio...  


In [6]:
df = pd.read_parquet("hf://datasets/VinitT/Sanskrit-Llama/data/train-00000-of-00001.parquet")
desired_data = df.drop("instruction", axis='columns')
desired_data = pd.concat([desired_data, csvFile],ignore_index=True)
print(desired_data.head(5))

c:\Users\ARYAN SURI\.conda\envs\py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


                                               input  \
0  ये त्रि॑ष॒प्ताः प॑रि॒यन्ति॒ विश्वा॑ रू॒पाणि॒ ब...   
1  पुन॒रेहि॑ वाचस्पते दे॒वेन॒ मन॑सा स॒ह । वसो॑ष्प...   
2  इ॒हैवाभि वि त॑नू॒भे आर्त्नी॑ इव॒ ज्यया॑ । वा॒च...   
3  उप॑हूतो वा॒चस्पति॒रुपा॒स्मान्वा॒चस्पति॑र्ह्वयत...   
4  वि॒द्मा श॒रस्य॑ पि॒तरं॑ प॒र्जन्यं॒ भूरि॑धायसम्...   

                                              output  
0  The three qualities of Rajogun, Tamogun and Sa...  
1  O Swami of speech Brahma Dev! Come to me with ...  
2  O Swami of speech Brahman! Just as by offering...  
3  We invoke Brahma, the swami of speech. May the...  
4  The root consciousness is the father of the cl...  


In [70]:
START_TOKEN = ''
PADDING_TOKEN = ''
END_TOKEN = ''

sanskrit_vocabulary = [
    # Vowels (Svara)
    "अ", "आ", "इ", "ई", "उ", "ऊ", "ऋ", "ॠ", "ऌ", "ॡ", "ए", "ऐ", "ओ", "औ",

    # Consonants (Vyanjana)
    # Gutturals (Kaṇṭhya)
    "क", "ख", "ग", "घ", "ङ",

    # Palatals (Tālavya)
    "च", "छ", "ज", "झ", "ञ",

    # Retroflex (Mūrdhanya)
    "ट", "ठ", "ड", "ढ", "ण",

    # Dentals (Dantya)
    "त", "थ", "द", "ध", "न",

    # Labials (Ōṣṭhya)
    "प", "फ", "ब", "भ", "म",

    # Semi-Vowels (Antastha)
    "य", "र", "ल", "व",

    # Sibilants (Ūṣman)
    "श", "ष", "स",

    # Aspirate (Visarga)
    "ह",

    # Additional Characters
    "ा", "ि", "ी", "ु", "ू", "ृ", "ॄ", "े", "ै", "ो", "ौ", "ं", "ः", "ँ"

    # Ligatures and Special Characters
    "क्ष", "त्र", "ज्ञ",
    PADDING_TOKEN, END_TOKEN
]


english_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', 
                        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                        ':', '<', '=', '>', '?', '@',
                        '[',
                        ']', '^', '_', '`', 
                        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                        'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 
                        'y', 'z', 
                        '{', '|', '}', '~', PADDING_TOKEN, END_TOKEN]

In [71]:
index_to_sanskrit = {k:v for k,v in enumerate(sanskrit_vocabulary)}
sanskrit_to_index = {v:k for k,v in enumerate(sanskrit_vocabulary)}
index_to_english = {k:v for k,v in enumerate(english_vocabulary)}
english_to_index = {v:k for k,v in enumerate(english_vocabulary)}

In [72]:
sanskrit_sentences = []
english_sentences = []
for i in range(len(desired_data)):
    if desired_data.iloc[i]["output"]!=None:
        english_sentences.append(desired_data.iloc[i]["output"].lower())
        sanskrit_sentences.append(desired_data.iloc[i]["input"])
len(sanskrit_sentences), len(english_sentences)

(20905, 20905)

In [73]:
import torch
from Transformer_model import Transformer
d_model = 512
batch_size = 30
ffn_hidden = 2048
num_heads = 8
drop_prob = 0.1
num_layers = 4
max_sequence_length = 200
sa_vocab_size = len(sanskrit_vocabulary)

transformer = Transformer(d_model, 
                          ffn_hidden,
                          num_heads, 
                          drop_prob, 
                          num_layers, 
                          max_sequence_length,
                          sa_vocab_size,
                          english_to_index,
                          sanskrit_to_index,
                          START_TOKEN, 
                          END_TOKEN, 
                          PADDING_TOKEN)

In [74]:
transformer

Transformer(
  (encoder): Encoder(
    (sentence_embedding): SentenceEmbedding(
      (embedding): Embedding(68, 512)
      (position_encoder): PositionalEncoding()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): SequentialEncoder(
      (0): EncoderLayer(
        (attention): MultiHeadAttention(
          (qkv_layer): Linear(in_features=512, out_features=1536, bias=True)
          (linear_layer): Linear(in_features=512, out_features=512, bias=True)
        )
        (norm1): LayerNormalization()
        (dropout1): Dropout(p=0.1, inplace=False)
        (ffn): PositionwiseFeedForward(
          (linear1): Linear(in_features=512, out_features=2048, bias=True)
          (linear2): Linear(in_features=2048, out_features=512, bias=True)
          (relu): ReLU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (norm2): LayerNormalization()
        (dropout2): Dropout(p=0.1, inplace=False)
      )
      (1): EncoderLayer(
        (attention): MultiHeadAt

In [75]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):

    def __init__(self, english_sentences, sanskrit_sentences):
        self.english_sentences = english_sentences
        self.sanskrit_sentences = sanskrit_sentences

    def __len__(self):
        return len(self.english_sentences)

    def __getitem__(self, idx):
        return self.english_sentences[idx], self.sanskrit_sentences[idx]

In [76]:
dataset = TextDataset(english_sentences, sanskrit_sentences)

In [77]:
len(dataset)

20905

In [78]:
dataset[1]

('o swami of speech brahma dev! come to me with a divine mind. o swami of life brahman! make me happy by giving me the desired results. give me the wisdom to wear the vedas i have studied. (2)',
 'पुन॒रेहि॑ वाचस्पते दे॒वेन॒ मन॑सा स॒ह । वसो॑ष्पते॒ नि र॑मय॒ मय्ये॒वास्तु॒ मयि॑ श्रु॒तम्  ॥ (२)')

In [79]:
train_loader = DataLoader(dataset, batch_size)
iterator = iter(train_loader)

In [80]:
for batch_num, batch in enumerate(iterator):
    print(batch)
    if batch_num > 3:
        break

[('the three qualities of rajogun, tamogun and satogun and earth, water, tej, air, sky, tanmatra and ego, the seven substances travel everywhere in divine form, brahma, the swami of speech, give me the divine power of those elements and substances. (1)', 'o swami of speech brahma dev! come to me with a divine mind. o swami of life brahman! make me happy by giving me the desired results. give me the wisdom to wear the vedas i have studied. (2)', 'o swami of speech brahman! just as by offering a bow string, both ends of it are stretched equally, in the same way, give me the power to wear vedashastra and the desired means of enjoyment. (3)', 'we invoke brahma, the swami of speech. may the brahmans invoked by us call us closer to us. we should always be equipped with complete knowledge and never get away. (4)', "the root consciousness is the father of the cloud arrow that nourishes all and the cloud that holds all is the father of the arrow and the earth with all the elements is the mother

In [81]:
from torch import nn
criterian = nn.CrossEntropyLoss(ignore_index=sanskrit_to_index[PADDING_TOKEN])

# When computing the loss, we are ignoring cases when the label is the padding token
for params in transformer.parameters():
    if params.dim() > 1:
        nn.init.xavier_uniform_(params)

optim = torch.optim.Adam(transformer.parameters(), lr=1e-4)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [82]:
NEG_INFTY = -1e9

def create_masks(eng_batch, kn_batch):
    num_sentences = len(eng_batch)
    look_ahead_mask = torch.full([max_sequence_length, max_sequence_length] , True)
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)
    encoder_padding_mask = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_self_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_cross_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)

    for idx in range(num_sentences):
      eng_sentence_length, kn_sentence_length = len(eng_batch[idx]), len(kn_batch[idx])
      eng_chars_to_padding_mask = np.arange(eng_sentence_length + 1, max_sequence_length)
      kn_chars_to_padding_mask = np.arange(kn_sentence_length + 1, max_sequence_length)
      encoder_padding_mask[idx, :, eng_chars_to_padding_mask] = True
      encoder_padding_mask[idx, eng_chars_to_padding_mask, :] = True
      decoder_padding_mask_self_attention[idx, :, kn_chars_to_padding_mask] = True
      decoder_padding_mask_self_attention[idx, kn_chars_to_padding_mask, :] = True
      decoder_padding_mask_cross_attention[idx, :, eng_chars_to_padding_mask] = True
      decoder_padding_mask_cross_attention[idx, kn_chars_to_padding_mask, :] = True

    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0)
    decoder_self_attention_mask =  torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
    decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)
    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask

In [ ]:
transformer.train()
transformer.to(device)
total_loss = 0
num_epochs = 10

for epoch in range(num_epochs):
    print(f"Epoch {epoch}")
    iterator = iter(train_loader)
    for batch_num, batch in enumerate(iterator):
        transformer.train()
        eng_batch, kn_batch = batch
        encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_masks(eng_batch, kn_batch)
        optim.zero_grad()
        sa_predictions = transformer(eng_batch,
                                     kn_batch,
                                     encoder_self_attention_mask.to(device), 
                                     decoder_self_attention_mask.to(device), 
                                     decoder_cross_attention_mask.to(device),
                                     enc_start_token=False,
                                     enc_end_token=False,
                                     dec_start_token=True,
                                     dec_end_token=True)
        labels = transformer.decoder.sentence_embedding.batch_tokenize(kn_batch, start_token=False, end_token=True)
        loss = criterian(
            sa_predictions.view(-1, sa_vocab_size).to(device),
            labels.view(-1).to(device)
        ).to(device)
        valid_indicies = torch.where(labels.view(-1) == sanskrit_to_index[PADDING_TOKEN], False, True)
        loss = loss.sum() / valid_indicies.sum()
        loss.backward()
        optim.step()
        #train_losses.append(loss.item())
        if batch_num % 100 == 0:
            print(f"Iteration {batch_num} : {loss.item()}")
            print(f"English: {eng_batch[0]}")
            print(f"Sanskrit Translation: {kn_batch[0]}")
            kn_sentence_predicted = torch.argmax(sa_predictions[0], axis=1)
            predicted_sentence = ""
            for idx in kn_sentence_predicted:
              if idx == sanskrit_to_index[END_TOKEN]:
                break
              predicted_sentence += index_to_sanskrit[idx.item()]
            print(f"Sanskrit Prediction: {predicted_sentence}")


            transformer.eval()
            kn_sentence = ("",)
            eng_sentence = ("should we go to the mall?",)
            for word_counter in range(max_sequence_length):
                encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, kn_sentence)
                predictions = transformer(eng_sentence,
                                          kn_sentence,
                                          encoder_self_attention_mask.to(device), 
                                          decoder_self_attention_mask.to(device), 
                                          decoder_cross_attention_mask.to(device),
                                          enc_start_token=False,
                                          enc_end_token=False,
                                          dec_start_token=True,
                                          dec_end_token=False)
                next_token_prob_distribution = predictions[0][word_counter] # not actual probs
                next_token_index = torch.argmax(next_token_prob_distribution).item()
                next_token = index_to_sanskrit[next_token_index]
                kn_sentence = (kn_sentence[0] + next_token, )
                if next_token == END_TOKEN:
                  break
            
            print(f"Evaluation translation (should we go to the mall?) : {kn_sentence}")
            print("-------------------------------------------")

INFO:tensorflow:Restoring parameters from ./checkpoints501/-12299
1.
--------------------------------
तस्मात् ते गत्वा तस्य प्रयोजनीयद्रव्याणि संग्रहीतुं शोमिरोणीयानां ग्रामं प्रविविशुः। <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
तस्मात् ते गत्वा तस्य प्रयोजनीयद्रव्याणि संग्रहीतुं शोमिरोणीयानां ग्रामं प्रविविशुः। <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Translation:  and they reasoned for every multitude had prayed <ukn> and they could not his brother was healed <ukn> his blood <ukn> him <ukn> him <ukn> him <ukn> 
   Expected:  and sent messengers before his face <ukn> and they went <ukn> and entered into a village of the samaritans <ukn> to make ready for him <ukn> 
--------------------------------
2.
--------------------------------
सर्वभूतस्थमात्मानं सर्वभूतानि चात्मनि <ukn> ईक्षते योगयुक्तात्मा सर्वत्र समदर्शन <ukn> 

In [ ]:
from hello import intro
from check import checkimages
from initialise_model import easy_model

intro()
finallist = checkimages()
model_result = easy_model(finallist)

'श्रीमद्धगवद्रीता अश्वत्थामा विकर्णश्च सोमदत्तिस्तथैव च ॥ १-८॥ (सोमदत्तिर्जयद्रथः) अन्ये च बहवः शूरा मदर्थे त्यक्तजीविताः । नानादाखप्रहरणाः सर्व युद्धविशारदाः ॥ १-९॥ अपर्याप्तं तदस्माकं बरं भीष्माभिरक्षितम्\u200c । पर्याप्तं त्विदमेतेषां बलं भीमाभिरक्षितम्\u200c ॥ १-१०॥ अयनेषु च सर्वेषु यथाभागमवस्थिताः । भीष्ममेवाभिरक्षन्तु भवन्तः सर्वं ए हि ॥ १-११॥ तस्य सञ्जनयन्हर्षं कुरुवृद्धः पितामहः । सिंहनादं विन्योचैः शद्ध दध्मौ प्रतापवान्\u200c ॥ १-१२॥ ततः शङ्खाश्च भेर्यश्च पणवानकगोमुखाः । सहसैवाभ्यहन्यन्त स शब्दस्तुमुखोऽभवत्\u200c ॥ १-१२॥ ततः शवतैर्येगयुक्ते महति स्यन्दने स्थितौ । माघवः पाण्डवश्चैव दिव्यौ शद्धौ प्रदध्मतुः ॥ १-१४॥ पाञ्चजन्यं हृषीकेशो देवदत्तं घनज्ञयः । पौण्डुं दध्मौ महारद्धं भीमकमा वृकोदरः ॥ १-१५॥ अनन्तविजयं राजा कुन्तीपुत्रो युधिष्ठिरः । नकुखः सहदेवश्च सुघोषमणिपुष्पको ॥ १-१६॥ कायश्च परमेष्वासः शिखण्डी च महारथः । धृष्टदयुनो विराटश्च सात्यकिश्चापराजितः ॥ १-१७॥ दरपदो द्रौपदेयाश्च सर्वशः पृथिवीपते । सौभद्रश्च महाबाहुः शङ्खान्दध्मुः पृथक्पृथक्\u200c ॥ १-१८॥ स घोषो धार्तराष्टाणां हृद

In [ ]:
transformer.eval()
def translate(eng_sentence):
  eng_sentence = (eng_sentence,)
  kn_sentence = ("",)
  for word_counter in range(max_sequence_length):
    encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, kn_sentence)
    predictions = transformer(eng_sentence,
                              kn_sentence,
                              encoder_self_attention_mask.to(device), 
                              decoder_self_attention_mask.to(device), 
                              decoder_cross_attention_mask.to(device),
                              enc_start_token=False,
                              enc_end_token=False,
                              dec_start_token=True,
                              dec_end_token=False)
    next_token_prob_distribution = predictions[0][word_counter]
    next_token_index = torch.argmax(next_token_prob_distribution).item()
    next_token = index_to_sanskrit[next_token_index]
    kn_sentence = (kn_sentence[0] + next_token, )
    if next_token == END_TOKEN:
      break
  return kn_sentence[0]
final_result = translate(model_result)
final_result

Ashwatthama and Somadatta. 1-8 (Somadattirjayadratha) and many others were left behind in the Shura. All of the warriors. 1-9 We are not strong enough, but we are strong enough. Sufficient power supply. 1-10 and all other conditions. Bhishma is everything. 1-11. his grandfather, who was twelve years old. The lion roared, and the lion roared. 1-12. and the other sheep and goats. And he was struck with the word. 1-12 in the Great Wall. The people of Magadha, the Pandavas, worshiped God. Verses 1-14, "Hrishikesh is the glory of God. Mahavir Mohapatra. 1-15: Yudhishthira, the son of King Kunti. The Divine and the Holy Spirit. 1-16 God is great and wise. Blessed are the mighty and the mighty. 1-17 Draupadi and all the earth. Saubhadra and Mahavishnu are separate entities. 1-18 He stirred up the hearts of the nations. The earth is not your bed, nor is it your bed. Galatians 1-19 - The flag of the nations with whom the law was made. Dhanurdayam Pandavas. 1-20 months. It's still up. January 18